In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
pip install transformers

     |████████████████████████████████| 3.4 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
     |████████████████████████████████| 596 kB 59.5 MB/s 
     |████████████████████████████████| 3.3 MB 42.4 MB/s 
     |████████████████████████████████| 61 kB 544 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertModel, AutoTokenizer, BertTokenizer
import re
import torch
df = pd.read_csv("/content/gdrive/MyDrive/tidydata.csv")
df['label'] = df['recommended'].apply(lambda x: 0 if x== True else 1)
X = df[['review']]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(df.index.values, df.label.values, test_size=0.2, random_state=40, stratify=df.label.values)
df['type'] = ['tmp']*df.shape[0]
df.loc[X_train, 'type'] = 'train'
df.loc[X_test, 'type'] = 'test'
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

X_train_list = list(df[df.type=='train'].review.values)
Y_train_list = list(df[df.type=='train'].label.values)
tmp1 = []
tmp2 = []
for i in range(len(X_train_list)):
    if X_train_list[i]==X_train_list[i]:
        tmp1.append(X_train_list[i])
        tmp2.append(Y_train_list[i])
X_train_list = tmp1
Y_train_list = tmp2

X_test_list = list(df[df.type=='test'].review.values)
Y_test_list = list(df[df.type=='test'].label.values)
tmp1 = []
tmp2 = []
for i in range(len(X_test_list)):
    if X_test_list[i]==X_test_list[i]:
        tmp1.append(X_test_list[i])
        tmp2.append(Y_test_list[i])
X_test_list = tmp1
Y_test_list = tmp2


In [4]:
# version 1
# please chose all version 1
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
train_ds = DataLoader(X_train_list, batch_size=16, num_workers=4, shuffle=False)
val_ds = DataLoader(X_test_list, batch_size=16, num_workers=4, shuffle=False)

In [ ]:
# version 2
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
)
encoded_data_train = tokenizer.batch_encode_plus(
    X_train_list,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_test_list,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(Y_train_list)
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(Y_test_list)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# version 2
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
train_ds = TensorDataset(input_ids_train, attention_masks_train, labels_train)
val_ds = TensorDataset(input_ids_val, attention_masks_val, labels_val)
train_ds = DataLoader(train_ds, batch_size=32, num_workers=4, shuffle=False)
val_ds = DataLoader(val_ds, batch_size=32, num_workers=4, shuffle=False)

In [ ]:
# version 2
from tqdm import tqdm
from transformers import BertModel, AutoTokenizer, BertTokenizer
import json
model = BertModel.from_pretrained(
    'bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

#model.train()
train_emb = []
for batch in tqdm(train_ds):
  batch = tuple(b.to(device) for b in batch)
  inputs = {
      'input_ids': batch[0],
      'attention_mask': batch[1],
  }
  train_emb += model(**inputs).pooler_output.tolist()
with open("/content/gdrive/MyDrive/train_embeddings.json", "w") as f:
    json.dump(train_emb, f)
    
#model.eval()
test_emb = []
for batch in tqdm(val_ds):
  batch = tuple(b.to(device) for b in batch)
  inputs = {
      'input_ids': batch[0],
      'attention_mask': batch[1],
  }
  test_emb += model(**inputs).pooler_output.tolist()
with open("/content/gdrive/MyDrive/test_embeddings.json", "w") as f:
    json.dump(test_emb, f)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda


100%|██████████| 1300/1300 [05:46<00:00,  3.76it/s]


In [5]:
# version 1
from tqdm import tqdm
from transformers import BertModel, AutoTokenizer, BertTokenizer
import json
model = BertModel.from_pretrained(
    'bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

#model.train()
train_emb = []
for batch in tqdm(train_ds):
  inputs = tokenizer(batch,padding=True,truncation=True,return_tensors='pt').to(device)
  train_emb += model(**inputs).pooler_output.tolist()
with open("/content/gdrive/MyDrive/train_embeddings.json", "w") as f:
    json.dump(train_emb, f)


#model.eval()
test_emb = []
for batch in tqdm(val_ds):
  inputs = tokenizer(batch,padding=True,truncation=True,return_tensors='pt').to(device)
  test_emb += model(**inputs).pooler_output.tolist()
with open("/content/gdrive/MyDrive/test_embeddings.json", "w") as f:
    json.dump(test_emb, f)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

cuda


100%|██████████| 2600/2600 [05:17<00:00,  8.19it/s]


In [ ]:
# version 3
from tqdm import tqdm
from transformers import BertModel, AutoTokenizer, BertTokenizer
import json
model = BertModel.from_pretrained(
    'bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

#model.train()
train_emb = []
for batch in tqdm(train_ds):
  inputs = tokenizer(batch,padding=True,truncation=True,return_tensors='pt').to(device)
  train_emb += model(**inputs).pooler_output.tolist()
with open("/content/gdrive/MyDrive/train_embeddings_2.json", "w") as f:
    json.dump(train_emb, f)


#model.eval()
test_emb = []
for batch in tqdm(val_ds):
  inputs = tokenizer(batch,padding=True,truncation=True,return_tensors='pt').to(device)
  test_emb += model(**inputs).pooler_output.tolist()
with open("/content/gdrive/MyDrive/test_embeddings_2.json", "w") as f:
    json.dump(test_emb, f)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

cuda


  0%|          | 0/5203 [00:00<?, ?it/s]


RuntimeError: ignored

In [5]:
# version Parametered BERT
from tqdm import tqdm
from transformers import BertModel, AutoTokenizer, BertTokenizer
from transformers import BertModel, BertConfig
configuration = BertConfig(num_attention_heads=12)
import json
model = BertModel(configuration)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

# version Parametered BERT
#model.train()
train_emb = []
for batch in tqdm(train_ds):
  inputs = tokenizer(batch,padding=True,truncation=True,return_tensors='pt').to(device)
  train_emb += model(**inputs).pooler_output.tolist()
with open("/content/gdrive/MyDrive/train_embeddings_heads12.json", "w") as f:
    json.dump(train_emb, f)


#model.eval()
test_emb = []
for batch in tqdm(val_ds):
  inputs = tokenizer(batch,padding=True,truncation=True,return_tensors='pt').to(device)
  test_emb += model(**inputs).pooler_output.tolist()
with open("/content/gdrive/MyDrive/test_embeddings_heads12.json", "w") as f:
    json.dump(test_emb, f)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

cuda


100%|██████████| 2600/2600 [05:25<00:00,  7.98it/s]


In [9]:
model1 = BertModel.from_pretrained(
    'bert-base-uncased')
model1==model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


False